In [13]:
import random

# --- Datos del problema ---
tasks = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
precedence = {
    'C': ['A'],
    'D': ['A'],
    'E': ['B'],
    'F': ['C', 'D'],
    'G': ['E', 'F'],
    'H': ['G'],
    'I': ['F'],
    'J': ['H', 'I']
}

def is_valid(order):
    position = {task: i for i, task in enumerate(order)}
    for task, preds in precedence.items():
        for pred in preds:
            if position[pred] > position[task]:
                return False
    return True

def fitness(order):
    position = {task: i for i, task in enumerate(order)}
    score = 100
    for task, preds in precedence.items():
        for pred in preds:
            if position[pred] > position[task]:
                score -= 30
            else:
                # Penalizar distancia grande entre pred y task
                score -= (position[task] - position[pred] - 1) * 2
    return score


## Generar población

In [14]:
def create_individual():
    return random.sample(tasks, len(tasks))

def create_population(size=20):
    return [create_individual() for _ in range(size)]

population = create_population()
for ind in population:
    print(ind, "-> Fitness:", fitness(ind))


['G', 'B', 'A', 'F', 'I', 'D', 'J', 'C', 'E', 'H'] -> Fitness: -92
['I', 'H', 'A', 'C', 'D', 'E', 'F', 'G', 'J', 'B'] -> Fitness: -26
['G', 'I', 'E', 'C', 'A', 'B', 'D', 'F', 'H', 'J'] -> Fitness: -86
['J', 'I', 'E', 'C', 'A', 'H', 'G', 'D', 'B', 'F'] -> Fitness: -132
['C', 'H', 'G', 'B', 'E', 'F', 'A', 'I', 'D', 'J'] -> Fitness: -78
['D', 'E', 'J', 'I', 'B', 'A', 'C', 'F', 'H', 'G'] -> Fitness: -108
['A', 'B', 'H', 'I', 'D', 'J', 'G', 'F', 'C', 'E'] -> Fitness: -94
['D', 'B', 'J', 'H', 'F', 'A', 'C', 'G', 'I', 'E'] -> Fitness: -110
['E', 'G', 'B', 'H', 'C', 'I', 'A', 'F', 'D', 'J'] -> Fitness: -74
['G', 'H', 'I', 'E', 'C', 'D', 'A', 'F', 'B', 'J'] -> Fitness: -112
['G', 'B', 'E', 'D', 'C', 'J', 'A', 'F', 'H', 'I'] -> Fitness: -106
['J', 'C', 'E', 'A', 'I', 'F', 'D', 'B', 'H', 'G'] -> Fitness: -138
['H', 'C', 'F', 'B', 'G', 'I', 'E', 'J', 'A', 'D'] -> Fitness: -44
['A', 'I', 'D', 'C', 'E', 'B', 'J', 'H', 'F', 'G'] -> Fitness: -60
['G', 'D', 'F', 'B', 'C', 'A', 'J', 'E', 'H', 'I'] -> Fi

## Operadores GA: selección, cruce, mutación

In [15]:
def selection(pop):
    a, b = random.sample(pop, 2)
    return a if fitness(a) > fitness(b) else b

def crossover(p1, p2):
    size = len(p1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None]*size
    child[start:end] = p1[start:end]
    fill = [x for x in p2 if x not in child]
    idx = 0
    for i in range(size):
        if child[i] is None:
            child[i] = fill[idx]
            idx += 1
    return child

def mutate(ind, prob=0.2):
    if random.random() < prob:
        i, j = random.sample(range(len(ind)), 2)
        ind[i], ind[j] = ind[j], ind[i]
    return ind


## Evolución

In [16]:
def evolve(population, generations=50):
    for gen in range(generations):
        new_pop = []
        for _ in range(len(population)):
            p1 = selection(population)
            p2 = selection(population)
            child = crossover(p1, p2)
            child = mutate(child)
            new_pop.append(child)
        population = new_pop
        best = max(population, key=fitness)
        print(f"Gen {gen+1:2d}: {best} | Fitness: {fitness(best)} | Válido: {is_valid(best)}")
    return best

best = evolve(population)
print("\nMejor solución encontrada:", best)


Gen  1: ['B', 'A', 'I', 'C', 'D', 'E', 'F', 'G', 'H', 'J'] | Fitness: 36 | Válido: False
Gen  2: ['B', 'I', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'J'] | Fitness: 38 | Válido: False
Gen  3: ['B', 'A', 'H', 'I', 'C', 'D', 'J', 'F', 'E', 'G'] | Fitness: -2 | Válido: False
Gen  4: ['I', 'A', 'C', 'B', 'E', 'D', 'F', 'H', 'J', 'G'] | Fitness: 2 | Válido: False
Gen  5: ['A', 'D', 'H', 'J', 'C', 'F', 'I', 'B', 'E', 'G'] | Fitness: 22 | Válido: False
Gen  6: ['H', 'A', 'D', 'J', 'C', 'F', 'I', 'B', 'E', 'G'] | Fitness: 22 | Válido: False
Gen  7: ['A', 'B', 'D', 'E', 'C', 'F', 'I', 'H', 'J', 'G'] | Fitness: 38 | Válido: False
Gen  8: ['A', 'D', 'C', 'B', 'E', 'H', 'F', 'I', 'J', 'G'] | Fitness: 38 | Válido: False
Gen  9: ['A', 'D', 'C', 'B', 'E', 'H', 'F', 'I', 'J', 'G'] | Fitness: 38 | Válido: False
Gen 10: ['A', 'B', 'D', 'E', 'C', 'H', 'F', 'I', 'J', 'G'] | Fitness: 34 | Válido: False
Gen 11: ['A', 'D', 'H', 'B', 'C', 'F', 'I', 'J', 'E', 'G'] | Fitness: 36 | Válido: False
Gen 12: ['A', 'H', 'B'